# Market depth (order book)


In [1]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=16)


<IB connected to 127.0.0.1:4001 clientId=16>

Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: cashfarm; usfuture; usfarm.nj; usfarm; usopt; euhmds; cashhmds; fundfarm; ushmds; secdefil.


To get a list of all exchanges that support market depth data and display the first five:


In [2]:
l = ib.reqMktDepthExchanges()
l[:5]


[DepthMktDataDescription(exchange='DTB', secType='OPT', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='LSEETF', secType='STK', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='SGX', secType='FUT', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='IDEALPRO', secType='CASH', listingExch='', serviceDataType='Deep', aggGroup=4),
 DepthMktDataDescription(exchange='ARCA', secType='STK', listingExch='', serviceDataType='Deep', aggGroup=2147483647)]

Let's subscribe to market depth data for EURUSD:


To see a live order book, an event handler for ticker updates is made that displays a dynamically updated dataframe:


In [3]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

ticker, contract = ticker_init(local_symbol=local_symbol)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


ticker.updateEvent += onTickerUpdate

IB.sleep(15)


,bidSize,bidPrice,askPrice,askSize
0,2.0,18644.0,18644.5,1.0
1,3.0,18643.75,18644.75,3.0
2,2.0,18643.5,18645.0,5.0
3,2.0,18643.25,18645.25,4.0
4,2.0,18643.0,18645.5,4.0


Stop the market depth subscription:


In [ ]:
ib.cancelMktDepth(contract)


,bidSize,bidPrice,askPrice,askSize
0,2.0,18643.75,18644.5,2.0
1,2.0,18643.5,18644.75,1.0
2,2.0,18643.25,18645.0,5.0
3,2.0,18643.0,18645.25,4.0
4,1.0,18642.75,18645.5,4.0


In [5]:
ib.disconnect()
